In [1]:
import pandas as pd
import numpy as np
import os
import openpyxl
import pickle
from bayespy.nodes import GaussianARD, Gamma
import re

In [2]:
with open('data_AC.pickle', 'rb') as f :
    data_AC = pickle.load(f)

In [3]:
data_AC.head(15)

,Port,Port_case,Category,Script_num,Language,Script,Move,Step,PS_num
0,울산(UL),1,투묘(AC),UL-AC-01-02,KR,항무울산 [SN] 감도 있습니까?,호출,응답,V1
1,울산(UL),1,투묘(AC),UL-AC-01-03,KR,네 [SN],호출,인사,S2
2,울산(UL),1,투묘(AC),UL-AC-01-04,KR,수고하십니다 본선 마이크 5번 묘에 23시 55분에 투묘 완료하였습니다 콜사인...,투묘,안내,V2
3,울산(UL),1,투묘(AC),UL-AC-01-05,KR,"네, 요청했습니다.",호출,인사,S3
4,울산(UL),2,투묘(AC),UL-AC-02-01,KR,네 감사합니다 수고하십시오,호출,개시,S1
5,울산(UL),2,투묘(AC),UL-AC-02-02,KR,항무울산 [SN] 감도 있습니까?,호출,문의,V1
6,울산(UL),2,투묘(AC),UL-AC-02-03,KR,네 어디시라구요?,호출,안내,S2
7,울산(UL),2,투묘(AC),UL-AC-02-04,KR,네 본선 [SN]입니다,호출,응답,V2
8,울산(UL),2,투묘(AC),UL-AC-02-05,KR,네 [SN],호출,안내,S3
9,울산(UL),2,투묘(AC),UL-AC-02-06,KR,투묘 보고 드리겠습니다 전 출항지 중국 대련 목적지 울산 OSCT 1번석 콜사...,접안,문의,V3


In [4]:
import pickle

In [5]:
from bs4 import BeautifulSoup

In [6]:
import requests

In [7]:
shipname_list = []
for i in range(1, 757) :
    url = requests.get('http://www.koswec.or.kr/koswec/information/shipsearch/selectShipSearchList.do?pageIndex=' + str(i))
    html = url.text
    soup = BeautifulSoup(html, 'html.parser')
    shipnames = soup.find_all('th', {'class' : 'tLeft pl_5'})
    for shipname in shipnames :
        shipname_list.append(shipname.text)

In [8]:
script_data = {'data' : data_AC[['Script_num', 'Script']], 'shipname_list' : shipname_list}

In [9]:
with open('script_data.pickle', 'wb') as f :
    pickle.dump(script_data, f)

In [4]:
data_AC = data_AC[data_AC.Step != '부정확']
data_AC = data_AC[data_AC.Step != '인사']

In [5]:
data_AC.Step.unique(), len(data_AC.Step.unique())

(array(['응답', '안내', '개시', '문의', '수신', '의도', '요청', '허가', '확인'], dtype=object),
 9)

In [6]:
ship_first = []
center_first = []
init = 'UL-AC-01'
temp_list = []
for row in data_AC.copy().iterrows() :
    if '-'.join(row[1][3].split('-')[:3]) != init :
        init = '-'.join(row[1][3].split('-')[:3])
        if row[1][8][0] == 'V' :
            center_first.append(temp_list)
        else :
            ship_first.append(temp_list)
        temp_list = []
    temp_list.append(row[1][7])
print(ship_first, center_first)

[['응답', '안내'], ['개시', '문의', '안내', '응답', '안내', '문의', '응답', '문의', '응답', '문의', '안내', '수신', '의도'], ['개시', '응답', '응답'], ['개시', '응답', '안내', '수신'], ['개시', '응답', '수신', '수신'], ['개시', '응답'], ['개시', '응답', '안내'], ['개시', '응답', '안내'], ['개시', '응답', '안내', '수신'], ['개시', '응답', '응답', '수신'], ['개시', '응답', '수신'], ['개시', '응답', '안내'], ['개시', '응답', '안내', '수신'], ['개시', '응답', '안내', '수신'], ['개시', '응답', '수신'], ['개시', '응답', '안내', '문의', '응답', '수신'], ['개시', '응답'], ['개시', '응답'], ['개시', '응답'], ['개시', '응답', '안내'], ['개시', '응답', '안내', '수신', '수신'], ['개시', '응답', '응답'], ['개시', '응답', '수신'], ['개시', '응답'], ['개시', '응답', '안내', '수신', '수신'], ['개시', '응답'], ['개시', '응답', '수신'], ['개시', '응답'], ['개시', '응답', '안내'], ['개시', '응답', '문의', '안내', '의도', '요청', '안내', '요청', '안내', '수신'], ['개시', '응답', '안내', '수신', '안내'], ['개시', '응답', '안내', '요청', '수신'], ['개시', '응답', '요청', '수신'], ['개시', '응답', '수신'], ['개시', '응답', '안내'], ['개시', '응답', '안내', '수신'], ['개시', '응답', '안내'], ['개시', '응답', '안내'], ['개시', '응답'], ['개시', '수신'], ['개시', '응답', '안내'], ['개시', '응답', '안내', '수신'

#### 일단 Classification이 되는지 부터

In [7]:
def clean_str(s):
    # only include alphanumerics and Korean
    s = re.sub(r"[^A-Za-z0-9가-힣(),!?\'\`]", " ", s)
    
    # insert spaces in words with apostrophes
    s = re.sub(r"\'s", " \'s", s)
    s = re.sub(r"\'ve", " \'ve", s)
    s = re.sub(r"n\'t", " n\'t", s)
    s = re.sub(r"\'re", " \'re", s)
    s = re.sub(r"\'d", " \'d", s)
    s = re.sub(r"\'ll", " \'ll", s)
    
    # 숫자라는 정보만 가져오기 위해 모든 숫자 0으로 통일
    s = re.sub(r'[0-9]', '0', s)
    
    # insert spaces in special characters
    s = re.sub(r",", " , ", s)
    s = re.sub(r"!", " ! ", s)
    s = re.sub(r"\(", " \( ", s)
    s = re.sub(r"\)", " \) ", s)
    s = re.sub(r"\?", " \? ", s)
    
    # only include alphanumerics and Korean again
    s = re.sub(r"[^A-Za-z0-9가-힣(),!?\'\`]", " ", s)
    
    # reduce multiple spaces to single spaces
    s = re.sub(r"\s{2,}", " ", s)
    return s.strip().lower()

In [8]:
raw_text = []
for row in data_AC.copy().iterrows() :
    raw_text.append(row[1][5])

In [9]:
raw_text

[' 항무울산 [SN] 감도 있습니까?',
 ' 수고하십니다 본선 마이크 5번 묘에  23시 55분에  투묘 완료하였습니다 콜사인 DCFH2입니다',
 ' 네 감사합니다 수고하십시오',
 ' 항무울산  [SN]   감도 있습니까?',
 ' 네 어디시라구요?',
 ' 네 본선 [SN]입니다',
 ' 네 [SN] ',
 ' 투묘 보고 드리겠습니다 전 출항지 중국 대련  목적지 울산 OSCT 1번석  콜사인 DSRC5  화물 가스오일 5천 톤  총 톤수 3901  델타라인 ETA 0100 되겠습니다.',
 ' 정해 1번 접안하신다고요?',
 ' 아니오. 원산 1부두입니다 OSCT 1번석 접안 예정입니다.',
 ' 아 원산 1부두요?',
 ' 네 맞습니다.',
 ' 아 콜사인 다시 한번요',
 ' 콜사인 DSRC5 입니다',
 ' 네 양지했습니다',
 ' 네 델타라인 통과할 때 다시 보고드리겠습니다',
 ' 울산VTS [SN]',
 ' Yes.',
 ' Thank you',
 ' 항무울산 [SN] 감도 있습니까?',
 ' 네, [SN]',
 ' 네, FUO(?)입니다. 00시 40분 마이크 2에 투묘했습니다.',
 ' 네, 수고하세요',
 ' 항무울산 [SN] 감도 있습니까?',
 ' 네, 수고 많으십니다. 본선 01시 55분 에코 2 지정 묘지에  투묘 하겠습니다.',
 ' 아, 죄송한데  저희 본선 E1에 투묘하겠습니다. 그 경계 부분이라서 잠시 헷갈렸습니다. 죄송합니다.',
 ' 네, 감사합니다. ',
 ' 항무울산 [SN] 감도 있습니까?',
 ' 네, 감사합니다.',
 ' 항무울산 [SN] 감도 있습니까?',
 ' [SN], 항무울산.',
 ' 네, 감사합니다. ',
 ' 항무울산 [SN] 감도 있습니까?',
 ' 네, [SN]',
 ' 네, 수고하십시오',
 ' 항무울산 [SN] 감도 있습니까?',
 ' 네, [SN]',
 ' 어…용현에서 투묘 하겠습니다. 이안할 때 보고 드리겠습니다.',
 ' 네, 감사합니다. ',
 ' 항무울산 항무울산. [SN]입니다

In [10]:
clean_text = [clean_str(x) for x in raw_text]

In [11]:
from konlpy.tag import Kkma
from konlpy.utils import pprint

In [12]:
raw_text

[' 항무울산 [SN] 감도 있습니까?',
 ' 수고하십니다 본선 마이크 5번 묘에  23시 55분에  투묘 완료하였습니다 콜사인 DCFH2입니다',
 ' 네 감사합니다 수고하십시오',
 ' 항무울산  [SN]   감도 있습니까?',
 ' 네 어디시라구요?',
 ' 네 본선 [SN]입니다',
 ' 네 [SN] ',
 ' 투묘 보고 드리겠습니다 전 출항지 중국 대련  목적지 울산 OSCT 1번석  콜사인 DSRC5  화물 가스오일 5천 톤  총 톤수 3901  델타라인 ETA 0100 되겠습니다.',
 ' 정해 1번 접안하신다고요?',
 ' 아니오. 원산 1부두입니다 OSCT 1번석 접안 예정입니다.',
 ' 아 원산 1부두요?',
 ' 네 맞습니다.',
 ' 아 콜사인 다시 한번요',
 ' 콜사인 DSRC5 입니다',
 ' 네 양지했습니다',
 ' 네 델타라인 통과할 때 다시 보고드리겠습니다',
 ' 울산VTS [SN]',
 ' Yes.',
 ' Thank you',
 ' 항무울산 [SN] 감도 있습니까?',
 ' 네, [SN]',
 ' 네, FUO(?)입니다. 00시 40분 마이크 2에 투묘했습니다.',
 ' 네, 수고하세요',
 ' 항무울산 [SN] 감도 있습니까?',
 ' 네, 수고 많으십니다. 본선 01시 55분 에코 2 지정 묘지에  투묘 하겠습니다.',
 ' 아, 죄송한데  저희 본선 E1에 투묘하겠습니다. 그 경계 부분이라서 잠시 헷갈렸습니다. 죄송합니다.',
 ' 네, 감사합니다. ',
 ' 항무울산 [SN] 감도 있습니까?',
 ' 네, 감사합니다.',
 ' 항무울산 [SN] 감도 있습니까?',
 ' [SN], 항무울산.',
 ' 네, 감사합니다. ',
 ' 항무울산 [SN] 감도 있습니까?',
 ' 네, [SN]',
 ' 네, 수고하십시오',
 ' 항무울산 [SN] 감도 있습니까?',
 ' 네, [SN]',
 ' 어…용현에서 투묘 하겠습니다. 이안할 때 보고 드리겠습니다.',
 ' 네, 감사합니다. ',
 ' 항무울산 항무울산. [SN]입니다

In [13]:
clean_text

['항무울산 sn 감도 있습니까 ?',
 '수고하십니다 본선 마이크 0번 묘에 00시 00분에 투묘 완료하였습니다 콜사인 dcfh0입니다',
 '네 감사합니다 수고하십시오',
 '항무울산 sn 감도 있습니까 ?',
 '네 어디시라구요 ?',
 '네 본선 sn 입니다',
 '네 sn',
 '투묘 보고 드리겠습니다 전 출항지 중국 대련 목적지 울산 osct 0번석 콜사인 dsrc0 화물 가스오일 0천 톤 총 톤수 0000 델타라인 eta 0000 되겠습니다',
 '정해 0번 접안하신다고요 ?',
 '아니오 원산 0부두입니다 osct 0번석 접안 예정입니다',
 '아 원산 0부두요 ?',
 '네 맞습니다',
 '아 콜사인 다시 한번요',
 '콜사인 dsrc0 입니다',
 '네 양지했습니다',
 '네 델타라인 통과할 때 다시 보고드리겠습니다',
 '울산vts sn',
 'yes',
 'thank you',
 '항무울산 sn 감도 있습니까 ?',
 '네 , sn',
 '네 , fuo ( ? ) 입니다 00시 00분 마이크 0에 투묘했습니다',
 '네 , 수고하세요',
 '항무울산 sn 감도 있습니까 ?',
 '네 , 수고 많으십니다 본선 00시 00분 에코 0 지정 묘지에 투묘 하겠습니다',
 '아 , 죄송한데 저희 본선 e0에 투묘하겠습니다 그 경계 부분이라서 잠시 헷갈렸습니다 죄송합니다',
 '네 , 감사합니다',
 '항무울산 sn 감도 있습니까 ?',
 '네 , 감사합니다',
 '항무울산 sn 감도 있습니까 ?',
 'sn , 항무울산',
 '네 , 감사합니다',
 '항무울산 sn 감도 있습니까 ?',
 '네 , sn',
 '네 , 수고하십시오',
 '항무울산 sn 감도 있습니까 ?',
 '네 , sn',
 '어 용현에서 투묘 하겠습니다 이안할 때 보고 드리겠습니다',
 '네 , 감사합니다',
 '항무울산 항무울산 sn 입니다',
 '네 , sn',
 '네 00시 드롭 앵커 완료했습니다',
 '네 , 감사합니다',
 '항무울산 sn 감도 있습니까 ?',
 '수고하

In [14]:
kkma = Kkma()

In [16]:
nouns_dict = []
for sentence in clean_text :
    nouns = list(filter(lambda x : len(x) > 1, kkma.nouns(sentence)))
    for noun in nouns :
        if noun not in nouns_dict :
            nouns_dict.append(noun)

In [17]:
nouns_dict

['항무',
 '항무울산',
 '울산',
 '감도',
 '수고',
 '마이크',
 '0번',
 '00',
 '00시',
 '00분',
 '투묘',
 '완료',
 '콜사인',
 '감사',
 '어디',
 '본선',
 '보고',
 '출항지',
 '중국',
 '대련',
 '목적지',
 '사인',
 '가스',
 '가스오일',
 '오일',
 '0천',
 '톤수',
 '0000',
 '델타',
 '델타라인',
 '라인',
 '접안',
 '원산',
 '0부두',
 '부두',
 '예정',
 '한번',
 '양지',
 '통과',
 '에코',
 '지정',
 '묘지',
 '저희',
 '경계',
 '부분',
 '용현',
 '이안',
 '드롭',
 '앵커',
 '000',
 '000분',
 '0앵커리지',
 '리지',
 '앵커리지',
 '배가',
 '위치',
 '조금',
 '선박번호',
 '번호',
 '00000',
 '이상',
 '오버',
 '양곡',
 '양곡부두',
 '태흥',
 '이드',
 '000000',
 '여기',
 '묘박지',
 '박지',
 '선박',
 '0묘지',
 '시간',
 '투묘시간',
 '드랍',
 '드랍앵커',
 '단독',
 '달포',
 '달포부두',
 '0부이',
 '부이',
 '순찰',
 '순찰지역',
 '지역',
 '접근',
 '도착',
 '당직',
 '투묘보고',
 '거영',
 '그쪽',
 '지금',
 '양묘',
 '자리',
 '좌현',
 '부탁',
 '호출',
 '호출부호',
 '부호',
 '숫자',
 '마이크0',
 '바우',
 '바우진',
 '입구',
 '시각',
 '외항',
 '묘박',
 '안쪽',
 '내일',
 '아침',
 '이동',
 '방파제',
 '앵커링',
 '남쪽',
 '호도',
 '옆쪽',
 '일본',
 '카시',
 '카시마',
 '에코0',
 '투묘하',
 '묘하',
 '톤수요',
 '수요',
 '0000톤',
 '얘기',
 '0000000000000000']

In [18]:
vocab_size = len(nouns_dict)
x_data = np.zeros([np.shape(clean_text)[0], vocab_size], dtype=np.float32)
for i, text in enumerate(clean_text):
    for j, vocab_word in enumerate(nouns_dict):
        if vocab_word in text:
            x_data[i][j] = 1

In [19]:
x_data

array([[ 1.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [20]:
x_data.shape

(405, 126)

In [21]:
labels = data_AC.Step.tolist()

In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
lbe = LabelEncoder()

In [24]:
y_data = lbe.fit_transform(labels)

Classifier fit

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
rfc = RandomForestClassifier()

In [27]:
rfc.fit(x_data, y_data)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
from sklearn.metrics import classification_report

In [29]:
yhat = rfc.predict(x_data)

In [30]:
print(classification_report(y_data, yhat))

             precision    recall  f1-score   support

          0       0.80      0.75      0.78       106
          1       1.00      0.32      0.48        19
          2       0.84      0.73      0.78        79
          3       0.50      0.81      0.62        85
          4       0.80      0.44      0.57         9
          5       0.86      0.72      0.78       103
          6       0.00      0.00      0.00         2
          7       1.00      1.00      1.00         1
          8       0.00      0.00      0.00         1

avg / total       0.76      0.72      0.72       405



/home/jin-lab/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
data_AC

#### Use bayesnet

In [ ]:
# from bayespy.inference import VB
# mu = GaussianARD(0, 1e-6)
# tau = Gamma(1e-6, 1e-6)
# y = GaussianARD(mu, tau, plates = (10, ))

In [ ]:
print(data_AC.loc[9].Script)